# Make the master table

In [9]:
import os 
import sys
import pandas as pd
import subprocess as sp
import glob
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')
hic_python = '/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python'

# default values for the command line
# sys.argv = [0] * 8
# sys.argv[1] =  'results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/'
# sys.argv[1] += 'BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
# sys.argv[2] = 'results/refs/ensembl/gencode.v19.annotation.bed'
# sys.argv[3] = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CM/FitHiChIP_L/FitHiChIP.interactions_FitHiC_Q0.01.bed'
# sys.argv[4] = 'results/refs/spp/SPP_D-Challenge_networks.xlsx'
# sys.argv[5] = 'results/refs/hg19/hg19.chrom.sizes'
# sys.argv[6] = 'results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/Monocyte.txt.gz'
# sys.argv[7] = 'results/main/loop_analysis/washU/'

# # parsing the commandline arguments
# coloc_fn = sys.argv[1]
# genes_fn = sys.argv[2]
# loop_fn = sys.argv[3]
# spp_fn = sys.argv[4]
# gs_fn = sys.argv[5]
# eqtl_fn = sys.argv[6]
# outdir = sys.argv[7]

# # setting the output file names
# os.makedirs(outdir, exist_ok=True)

In [10]:
# %load ./scripts/loop_analysis/Annotation_SNPs_with_Surround_Genes_Loops.sh
#!/usr/bin/env python

# In[1]:


# Commandline Specification
# Input
# ----------------------------------------------
# 1) colocalization file  x
# 2) gencode gene annotation file in bed format x
# 3) loop data x 
# 4) spp network data x 
# 5) reference chrom sizes x 
# Output
# ----------------------------------------------
# 1) snp-gene loop summary file
# 2) snp-gene pairs longrange file with index
# 3) snp-gene loops longrange file with index

# default values for the command line
#sys.argv = [0] * 7
#sys.argv[1] =  'results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/'
#sys.argv[1] += 'BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
#sys.argv[2] = 'results/refs/ensembl/gencode.v19.annotation.bed'
#sys.argv[3] = 'results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CM/FitHiChIP_L/FitHiChIP.interactions_FitHiC_Q0.01.bed'
#sys.argv[4] = 'results/refs/spp/SPP_D-Challenge_networks.xlsx'
#sys.argv[5] = 'results/refs/hg19/hg19.chrom.sizes'
#sys.argv[6] = 'results/main/loop_analysis/washU/'
script = "scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops.py"
script = "scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py"
gs = "results/refs/hg19/hg19.chrom.sizes"
gencode = "results/refs/ensembl/gencode.v19.annotation.bed"
spp = "results/refs/spp/SPP_D-Challenge_networks.xlsx"

# template file names for colocalization, hichip, eqtl and outdir
coloc_tmp = "results/main/2021_Nikhil_eQTL/Results/Colocalization/{gwas}/{eqtl}_{cline}/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed"
fithichip_tmp = "results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/{cline}/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed"
eqtl_tmp = "results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/{eqtl}/{cline}.txt.gz"
outdir_tmp = "results/main/loop_analysis/Coloc_Approach/{gwas}_test/{dice_cline}/{eqtl_study}/{eqtl_cline}/"

# getting a list of DICE clines with HiChIP data
fithichip_clines = glob.glob("results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/*")
fithichip_clines = [x.split('/')[-1] for x in fithichip_clines]

# eqtl mapper
eqtl_mapper = pd.read_table('results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/eqtl.cline.mapper.study.tsv')
eqtl_mapper.sort_values(['will_analyze', 'study', 'dice_shortname'], ascending=[False, True, True], inplace=True)
eqtl_mapper[['study', 'dice_shortname', 'will_analyze', 'eqtl_file']]
eqtl_mapper = eqtl_mapper.loc[eqtl_mapper['will_analyze'] == 'Yes']
eqtl_mapper = eqtl_mapper.loc[eqtl_mapper.dice_shortname.isin(fithichip_clines)]
eqtl_clines = eqtl_mapper['eqtl_file'].str.split('/')
eqtl_clines = [x[-1].split('.')[0] for x in eqtl_clines]
eqtl_mapper['eqtl_cline'] = eqtl_clines
eqtl_mapper = eqtl_mapper[['study', 'dice_shortname', 'eqtl_cline', 'eqtl_file']]

# pchic dataset
pchic_fn = 'results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv'

In [11]:
eqtl_mapper.head()

,study,dice_shortname,eqtl_cline,eqtl_file
0,BLUEPRINT_eQTL,CM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
1,BLUEPRINT_eQTL,NCM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
2,BLUEPRINT_eQTL,TFH,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
3,BLUEPRINT_eQTL,TH1,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
4,BLUEPRINT_eQTL,TH17,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...


In [12]:
# get gwas runs
gwas_runs = [os.path.basename(x) for x in glob.glob('results/main/2021_Nikhil_eQTL/Data/T1D_GWAS/*')]
gwas_runs = gwas_runs[2:3]

# get eqtl runs 
eqtl_runs = [x for x in glob.glob('results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/*/*')]
eqtl_runs = [x for x in eqtl_runs if x != 'README' and 'eQTL' in x]

In [13]:
eqtl_mapper.head()

,study,dice_shortname,eqtl_cline,eqtl_file
0,BLUEPRINT_eQTL,CM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
1,BLUEPRINT_eQTL,NCM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
2,BLUEPRINT_eQTL,TFH,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
3,BLUEPRINT_eQTL,TH1,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
4,BLUEPRINT_eQTL,TH17,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...


In [14]:
# qscripts_dir = 'results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton/qjobs/'
# os.makedirs(qscripts_dir, exist_ok=True)

In [15]:
eqtl_mapper

,study,dice_shortname,eqtl_cline,eqtl_file
0,BLUEPRINT_eQTL,CM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
1,BLUEPRINT_eQTL,NCM,Monocyte,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
2,BLUEPRINT_eQTL,TFH,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
3,BLUEPRINT_eQTL,TH1,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
4,BLUEPRINT_eQTL,TH17,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
5,BLUEPRINT_eQTL,TH2,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
6,BLUEPRINT_eQTL,THSTAR,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
7,BLUEPRINT_eQTL,TREGMEM,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
8,BLUEPRINT_eQTL,TREGNAIVE,T-cell,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...
23,DICE_eQTL,CD4N,CD4_NAIVE,results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_s...


In [16]:
for gwas in gwas_runs:
    for i, sr in eqtl_mapper.iterrows():
        
        coloc = coloc_tmp.format(gwas=gwas, eqtl=sr.study, cline=sr.eqtl_cline)
        fithichip = fithichip_tmp.format(cline=sr.dice_shortname)
        outdir = outdir_tmp.format(gwas=gwas, 
                                   dice_cline=sr.dice_shortname,
                                   eqtl_cline=sr.eqtl_cline,
                                   eqtl_study=sr.study)
        
        
        # Changing the celltype naming for BLUEPRINT monocytes in order to include
        # Chicago scores from PC HiC
        if sr.eqtl_cline == 'Monocyte':
            celltype = 'CLASSICAL_MONOCYTES'
        else:
            celltype = sr.eqtl_cline
            
        cmd = "{} {} {} {} {} {} {} {} {} {} {}".format(hic_python, script, coloc, gencode, 
                                                  fithichip, spp, gs, sr.eqtl_file,
                                                  pchic_fn, sr.eqtl_cline, outdir)  
        
        print(cmd)
        
        if not os.path.exists(outdir):
            
            #             script_fn = '{}_{}_{}_{}.qscript'.format(gwas, sr.dice_shortname, sr.eqtl_cline, sr.study)
            #             script_fn = os.path.join(qscripts_dir, script_fn)
            #             with open(script_fn, 'w') as fw:
            #                 fw.write(cmd)
                
            job = sp.Popen(cmd, shell=True, stderr=sp.PIPE, stdout=sp.PIPE)
            out, err = job.communicate()
            print('Now running: {}'.format(cmd))
            print(out.decode())
            print(err.decode())
            print('Results can now be found at: {}'.format(outdir))
        else:
            msg = 'Previously ran {}'.format(outdir)
            print(msg)
            
        print()
        

/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/CM/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/Monocyte.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv Monocyte results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/CM/BLUEPRINT_eQTL/Monocyte/
Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_Monocyte

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_Monocyte/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/NCM/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/Monocyte.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv Monocyte results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/NCM/BLUEPRINT_eQTL/Monocyte/
There are 15 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 635 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 103342 significant loops.
There are 85 SNP-Gene pairs with a loop.
Ma

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_T-cell/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TFH/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/T-cell.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv T-cell results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TFH/BLUEPRINT_eQTL/T-cell/
There are 16 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 613 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 46172 significant loops.
There are 28 SNP-Gene pairs with a loop.
Master is s

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_T-cell/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TH1/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/T-cell.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv T-cell results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TH1/BLUEPRINT_eQTL/T-cell/
There are 16 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 613 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 63241 significant loops.
There are 61 SNP-Gene pairs with a loop.
Master is s

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/BLUEPRINT_eQTL_T-cell/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TH17/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/BLUEPRINT_eQTL/T-cell.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv T-cell results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TH17/BLUEPRINT_eQTL/T-cell/
There are 16 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 613 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 76270 significant loops.
There are 70 SNP-Gene pairs with a loop.
Master is

...

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/DICE_eQTL_TH17/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TH17/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/DICE_eQTL/TH17.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv TH17 results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TH17/DICE_eQTL/TH17/
There are 7 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 281 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 76270 significant loops.
There are 33 SNP-Gene pairs with a loop.
Master is starting with 281 snp-g

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/DICE_eQTL_TH2/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TH2/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/DICE_eQTL/TH2.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv TH2 results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TH2/DICE_eQTL/TH2/
There are 7 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 291 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 58115 significant loops.
There are 10 SNP-Gene pairs with a loop.
Master is starting with 291 snp-gene pa

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/DICE_eQTL_TH1-17/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/THSTAR/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/DICE_eQTL/TH1-17.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv TH1-17 results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/THSTAR/DICE_eQTL/TH1-17/
There are 5 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 204 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 50757 significant loops.
There are 20 SNP-Gene pairs with a loop.
Master is starting wi

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/DICE_eQTL_TREG_MEMORY/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TREGMEM/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/DICE_eQTL/TREG_MEMORY.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv TREG_MEMORY results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TREGMEM/DICE_eQTL/TREG_MEMORY/
There are 10 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 407 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 56567 significant loops.
There are 31 SNP-Gene pairs with a loop

Now running: /mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/python scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops_V2.py results/main/2021_Nikhil_eQTL/Results/Colocalization/T1D_34012112_Gaulton/DICE_eQTL_TREG_NAIVE/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed results/refs/ensembl/gencode.v19.annotation.bed results/main/2021_Nikhil_eQTL/Data/FitHiChIP_Loops/TREGNAIVE/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed results/refs/spp/SPP_D-Challenge_networks.xlsx results/refs/hg19/hg19.chrom.sizes results/main/2021_Nikhil_eQTL/Data/eqtl_sqtl_summ_stats/DICE_eQTL/TREG_NAIVE.txt.gz results/main/pc_hic/data/PCHiC_peak_matrix_cutoff5.tsv TREG_NAIVE results/main/loop_analysis/Coloc_Approach/T1D_34012112_Gaulton_test/TREGNAIVE/DICE_eQTL/TREG_NAIVE/
There are 6 colocalized SNP-gene pairs
There are 57774 genes in this GTF-derived file.
There are 321 colocalized snp-gene pairs within +/- 5kb.
FitHiChIP found 51613 significant loops.
There are 23 SNP-Gene pairs with a loop.